# Part 0 - Utils

## Import Libraries

In [ ]:
%%capture
import os
import time

import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt

import kagglehub
import scipy.io

!pip install ipython-autotime
%load_ext autotime

import tensorflow as tf

## Dataset <font color='red'>GET READY FOR SUBMISSION
Cars196 - 196 different classes of vehicles

### Download Data

In [ ]:
#@title Download from kaggle
download_from_kaggle = False #@param {type:"boolean"}
# Download latest version
if download_from_kaggle:
    path = kagglehub.dataset_download("jessicali9530/stanford-cars-dataset")

In [ ]:
#@title Move data to new directory
!mkdir /content/dataset
if download_from_kaggle:
    !mv /root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/* /content/dataset

In [ ]:
#@title Create and set data path - connect to drive if necessary

#@markdown > We suggest to use this as the path to the directory containing this notebook
PATH = '/content/drive/MyDrive/Colab Notebooks/ImageMulticlassClassification/' #@param {"type":"string"}

use_drive_path = True #@param {type:"boolean"}
if use_drive_path:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown > The default path if you don't use the drive path is `/content/dataset/`

# Set data directory path
data_dir = PATH + 'dataset/' if use_drive_path else '/content/dataset/'

# Make sure to create the path if it's missing
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)

print(f'{data_dir=}')

In [ ]:
#@title Copy data to drive
#@markdown the data will be copied to `data_dir`
copy_data_to_data_dir = False #@param {type:"boolean"}
if copy_data_to_data_dir:
    os.system(f"!cp -r /content/dataset/* {data_dir}")

In [ ]:
#@title Get Annotations File from google drive <font color='red'> - CHANGE FILE ID (הוא מסביר בסרטון בדקה 13 איך עושים את זה)
!gdown 1Mmcu4btdVorH9S6QH6qbhTl1UrVx217o # Change this id

metadata_file_path = data_dir + 'stanford_cars_with_class_names.xlsx'

os.system(f'!mv /content/stanford_cars_with_class_names.xlsx {metadata_file_path}')


## Load MetaData

Extract image names, class names and numbers

In [ ]:
#@title Load metadata for train and test sets <font color='red'> CHANGE PATH

# Take the last 3 columns with image name, class name and class number
train_annotations = pd.read_excel(metadata_file_path, sheet_name='train').iloc[:, -1:-4:-1]
test_annotations = pd.read_excel(metadata_file_path, sheet_name='test').iloc[:, -1:-4:-1]

# For some reason the labels in the test annotations file load with additional unnecessary \'\s
test_annotations.image = test_annotations.image.str.strip("'")

In [ ]:
#@title Create the Models dir if not existing <font color='red'> - change `models_dir` as needed
models_dir = PATH + 'models/'
# models_dir = '/content/models/'


# Create the directory if it doesn't exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [ ]:
#@title Dowload and unzip ZIP from Drive <font color="red"> - You need to set this up
#@markdown Should contain `dataset` and `models` directories
!gdown #<YOURZIPFILEID>
!unzip #<YOURZIPFILE>.zip
print(os.listdir())

## Plot Single Example

In [ ]:
#@title Helper functions
%%script echo skipping
def get_class_and_model(image_name: str, annotations: pd.DataFrame):
    """
    Get the class number and model name for a given image name.

    Args:
        image_name (str): The image file name (e.g., "00001.jpg").
        annotations: Annotations from the .mat file.

    Returns:
        tuple: (class_number, model_name) if found, otherwise raises an error.
    """
    if image_name not in annotations.image.values:
        raise ValueError(f"Image {image_name} not found in annotations.")

    class_number = annotations[annotations.image == image_name].values[0,1]
    model_name = annotations[annotations.image == image_name].values[0,2]
    return class_number, model_name

def show_image_with_title(image_name, dataset_path, annotations):
    """
    Display an image with its class and model name as the title.

    Args:
        image_name (str): The name of the image file (e.g., "car_ims/000001.jpg").
        dataset_path (str): Path to the dataset containing the images.
        annotations: Annotations from the .mat file.
        class_names_list: List of class names extracted from the .mat file.
    """
    # Get class number and model name for the image
    class_number, model_name = get_class_and_model(image_name, annotations)

    # Construct the title
    title = f"Class {class_number} → \"{model_name}\""
    updated_img_path = image_name.replace("car_ims/0", "")
    # Load and display the image
    image_path = os.path.join(dataset_path, updated_img_path)
    try:
        image = Image.open(image_path)
        plt.imshow(image)
        plt.axis("off")
        plt.title(title)
        plt.show()
    except FileNotFoundError:
        print(f"Image file not found: {image_path}")

### Example usage


*   **Image Name**: car_ims/000001.jpg
*   **Class Number**: 1
*   **Model Name**: "AM General Hummer SUV 2000"

In [ ]:
#@title Extract metadata
%%script echo skipping
example_image_name = train_annotations.loc[0,'image']
example_class, example_model = get_class_and_model(example_image_name, train_annotations)

example_image_name, example_class, example_model

In [ ]:
#@title Show example
%%script echo skipping
dataset_path = data_dir + "cars_train/cars_train"
show_image_with_title(example_image_name, dataset_path, train_annotations)

---
# Part 1 - Pre-processing and general set-up <font color="red"> - check befor running parts 2 to 4

you can also choose partial data for code testing purposes

## 1.1 Define hyperparameters

Here we define important settings (hyperparameters) for training our deep learning models.
> `IMG_SIZE` sets the image size to 224 pixels.
>
>`NUM_CLASSES` specifies 196 different car types.
>
>`BATCH_SIZE` sets the training batch size to.
>
>`EPOCHS` sets the number of training runs.
>
>`VAL_SPLIT` fraction of the datato allocate for validation.

These settings influence how the model learns and performs.

In [ ]:
IMG_SIZE = 224  #@param # EfficientNetV2L expects 224x224 images
NUM_CLASSES = 196 #@param
BATCH_SIZE = 16 #@param {"type":"integer"}
EPOCHS = 3 #@param {"type":"integer"}
VAL_SPLIT = 0.2 #@param {"type":"number"}


## 1.2 Define Data augmentation layers




The data augmentation transformations I used:

1.  **`tf.keras.layers.RandomFlip("horizontal")`**: This layer randomly flips images horizontally.  Half the time the image will be flipped, and half the time it will not.  This helps the model learn that an object is the same regardless of its left-right orientation.

2.  **`tf.keras.layers.RandomRotation(0.2)`**: This layer randomly rotates images by up to 20% of a full rotation (0.2 * 360 degrees = 72 degrees).  So, the model will see slightly rotated versions of the images. This makes the model robust to small variations in the angle of the object in an image.

3.  **`tf.keras.layers.RandomTranslation(0.2, 0.2)`**: This layer randomly translates images both horizontally and vertically by up to 20% of the image size. This shifts the image slightly, making the model less sensitive to the exact position of the object.

4.  **`tf.keras.layers.RandomZoom(0.2)`**:  This layer randomly zooms into images by up to 20%. This introduces different levels of zoom, forcing the model to generalize to images with different scales of objects.

5.  **`tf.keras.layers.RandomBrightness(0.2)`**: This layer randomly adjusts the brightness of images by up to 20%. This augmentation helps the model to be less sensitive to changes in lighting conditions.

6.  **`tf.keras.layers.RandomContrast(0.2)`**: This layer randomly adjusts the contrast of images by up to 20%.  Similar to brightness augmentation, this makes the model more robust to variations in image contrast.


In essence, these transformations create slightly altered versions of the original images, which helps train a more robust model that performs better on unseen data and is less prone to overfitting.  The model learns features that are invariant to small changes in orientation, position, scale, brightness and contrast.


In [ ]:
#@title Augmentations
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

## 1.3 Process Datasets

In [ ]:
#@title Load dataset functions
def vectorize_label(index, length):
    """
    Creates a TensorFlow tensor of zeros with length 'length', and sets the element at 'index' to 1.
    Args:
        length: The desired length of the tensor.
        index: The index of the element to set to 1.
    Returns:
        A TensorFlow tensor of shape (length,) with a 1 at the specified index and 0 elsewhere.
    """
    if index < 0 or index >= length:
        print("Error: Index out of bounds.")
        return None
    tensor = tf.zeros(length, dtype=tf.int32)
    tensor = tf.tensor_scatter_nd_update(tensor, [[index]], [1])
    return tensor


def load_and_preprocess_image(path, label):

    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)

    return image, label


def create_dataset_from_directory_and_metadata(data_dir, annotations, img_size, batch_size, val_split=0) -> tf.data.Dataset:
    # create image names and labels lists
    image_paths = list(data_dir + annotations.values[:,0])
    labels = annotations.values[:,2] - 1 # Make the first label 0 and the last is 195
    labels = [vectorize_label(label, NUM_CLASSES) for label in labels]

    # import pdb ; pdb.set_trace()

    # create tf dataset object
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # pre-split processing
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    # dataset = dataset.cache()
    dataset = dataset.shuffle(buffer_size=len(image_paths))

    if val_split > 0:
        # Split dataset
        val_size = int(len(image_paths) * val_split)
        train_dataset = dataset.skip(val_size)
        val_dataset = dataset.take(val_size)

        # Batch datasest
        train_dataset = train_dataset.batch(batch_size)
        val_dataset = val_dataset.batch(batch_size)

        return train_dataset, val_dataset
    else:
        return dataset.batch(batch_size)


In [ ]:
#@title Get train, validation and test datasets
train_ds, val_ds = create_dataset_from_directory_and_metadata(
    data_dir + 'cars_train/cars_train/',
    train_annotations, IMG_SIZE, BATCH_SIZE, VAL_SPLIT)

test_ds = create_dataset_from_directory_and_metadata(
    data_dir + 'cars_test/cars_test/',
    test_annotations, IMG_SIZE, BATCH_SIZE)


In [ ]:
#@title Prepare datasets for model training and evaluation
# Apply data augmentation to training set
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Normalize pixel values
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Optimize performance
prefetch = False #@param {"type":"boolean"}
if prefetch:
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

# Print dataset sizes
print(f"Training samples: {len(train_ds) * BATCH_SIZE}")
print(f"Validation samples: {len(val_ds) * BATCH_SIZE}")
print(f"Test samples: {len(test_ds) * BATCH_SIZE}")


In [ ]:
#@title Take a subset of the data <font color='red'> - for code testing purposes
take_partial_data = False #@param {type:"boolean"}
how_many_batches = 3 #@param {"type":"integer"}

if take_partial_data:
    train_ds = train_ds.take(how_many_batches)
    val_ds = val_ds.take(how_many_batches)
    test_ds = test_ds.take(how_many_batches)

## 1.4 Get base model for Transfer learning and embedding

I chose EfficientNetV2L with `imagenet` weights as the base model

It offers:

- High accuracy and efficiency
- Pre-trained knowledge from ImageNet
- Feature extraction capabilities
- Faster training
- Good performance in transfer learning scenarios

In [ ]:
#@title Get Base Model
from tensorflow.keras.applications import EfficientNetV2L

donload_new_base_model = True #@param {type:"boolean"}
#@markdown make sure you have a `"base_model.keras"` file in `models_dir` if you set the following True
load_from_models_dir = False #@param {type:"boolean"}



if donload_new_base_model:
    # Load the pre-trained model
    base_model = EfficientNetV2L(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3))
elif load_from_models_dir:
    base_model = tf.keras.models.load_model(models_dir + '/base_model.keras')
else:
    raise ValueError("Please set either `donload_new_base_model` or `load_from_models_dir` to True")

# Freeze the base model layers
base_model.trainable = False

## 1.5 Define evaluation functions

In [ ]:
#@title Single Model evaluation functions
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def plot_accuracy_along_training(history):
    """
    Plots the training and validation accuracy curves from a Keras model's history.

    Args:
        history: The history object returned by the Keras model's fit method.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

def test_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro') # Use 'macro' for multi-class
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    return accuracy, precision, recall, f1

# Assuming y_test and y_pred are already defined from your model predictions on the test set
def test_DL_model(model, test_ds):
    y_pred = model.predict(test_ds)
    y_pred = np.argmax(y_pred, axis=1) # Convert probabilities to class labels

    y_true = []
    for images, labels in test_ds:
        y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_true = np.array(y_true)

    return test_model(y_true, y_pred)

In [ ]:
#@title Compare experiment functions
def plot_comparison(histories, metrics):

    plt.figure(figsize=(12, 6))

    for i, history in enumerate(histories):
        for metric in metrics:
            plt.plot(history.history[metric], label=f'Experiment {i+1} - {metric}')

    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Comparison of Training Histories')
    plt.legend()
    plt.grid(True)
    plt.show()


def compare_test_results(results):
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    df = pd.DataFrame(results, columns=metrics)
    df.index = [f'Experiment {i+1}' for i in range(len(results))]
    return df



In [ ]:
#@title Save model function
def save_tf_model(model, model_dir, model_name):
    """Saves a TensorFlow model to a specified directory.

    Args:
        model: The TensorFlow model to save.
        model_dir: The directory to save the model in.
        model_name: The name to give the saved model.
    """

    model_path = os.path.join(model_dir, model_name)
    model.save(model_path)
    print(f"Model saved to: {model_path}")


---
# Part 2 - Transfer Learning Experiments

## Configuration Analysis


The Transfer Learning experiments explore different architectures built on top of EfficientNetV2L, focusing on the trade-off between model capacity and regularization:

1. **Base Configuration (Experiment 1)**:
   - Single dense layer (1024 units) with moderate dropout (0.5)
   - Balanced approach between complexity and regularization
   - Serves as baseline for comparison
   - Minimalist architecture to test if simple feature mapping is sufficient

2. **Enhanced Architecture (Experiment 2)**:
   - Two dense layers (1024 units each) with dropout (0.5) between them
   - Increased model capacity for more complex feature relationships
   - Additional layer allows hierarchical feature learning
   - Higher parameter count but potential for better feature abstraction

3. **High Regularization (Experiment 3)**:
   - Single dense layer (1024 units) with aggressive dropout (0.8)
   - Tests impact of stronger regularization
   - Focuses on preventing overfitting
   - Same capacity as base model but more conservative in feature utilization

Key differences:
- Experiment 1 vs 2: Tests if additional capacity improves performance
- Experiment 2 vs 3: Compares complex architecture against stronger regularization
- Experiment 1 vs 3: Evaluates impact of dropout strength on same architecture

## 2.1 Set-up


In [ ]:
#@title Imports
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

## 2.2 Expirement 1

In [ ]:
#@title Define Model
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
tl1_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
tl1_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
tl1_model.summary()

In [ ]:
#@title Load a pretrained model
#@markdown Loading from the set `models_dir`
# Load the pre-trained model
model_name = "tl1_model" # @param {"type":"string"}
model_path = models_dir + f'/{model_name}.keras' # Replace with the actual path to your saved model
tl1_model = tf.keras.models.load_model(model_path)


In [ ]:
#@title Train the model
tl1_history = tl1_model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)


In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(tl1_history)

In [ ]:
#@title Test Model
accuracy_1, precision_1, recall_1, f1_1 = test_DL_model(tl1_model, test_ds)

print(f"Test Accuracy: {accuracy_1}")
print(f"Test Precision: {precision_1}")
print(f"Test Recall: {recall_1}")
print(f"Test F1-score: {f1_1}")

In [ ]:
#@title Save model
model_name = "tl1_model" #@param {"type":"string"}
save_tf_model(tl1_model, models_dir, f'{model_name}.keras')

## 2.3 Expirement 2

In [ ]:
#@title Define Model
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
tl2_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
tl2_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
tl2_model.summary()

In [ ]:
#@title Load a pretrained model
#@markdown Loading from the set `models_dir`
# Load the pre-trained model
model_name = "tl2_model" # @param {"type":"string"}
model_path = models_dir + f'/{model_name}.keras' # Replace with the actual path to your saved model
tl2_model = tf.keras.models.load_model(model_path)


In [ ]:
#@title Train the model
tl2_history = tl2_model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)


In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(tl2_history)

In [ ]:
#@title Test Model
accuracy_2, precision_2, recall_2, f1_2 = test_DL_model(tl2_model, test_ds)

print(f"Test Accuracy: {accuracy_2}")
print(f"Test Precision: {precision_2}")
print(f"Test Recall: {recall_2}")
print(f"Test F1-score: {f1_2}")

In [ ]:
#@title Save model
model_name = "tl2_model" #@param {"type":"string"}
save_tf_model(tl2_model, models_dir, f'{model_name}.keras')

## 2.4 Expirement 3

In [ ]:
#@title Define Model
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.8)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
tl3_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
tl3_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
tl3_model.summary()

In [ ]:
#@title Load a pretrained model
#@markdown Loading from the set `models_dir`
# Load the pre-trained model
model_name = "tl3_model" # @param {"type":"string"}
model_path = models_dir + f'/{model_name}.keras' # Replace with the actual path to your saved model
tl3_model = tf.keras.models.load_model(model_path)


In [ ]:
#@title Train the model
tl3_history = tl3_model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)

In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(tl3_history)

In [ ]:
#@title Test Model
accuracy_3, precision_3, recall_3, f1_3 = test_DL_model(tl3_model, test_ds)

print(f"Test Accuracy: {accuracy_3}")
print(f"Test Precision: {precision_3}")
print(f"Test Recall: {recall_3}")
print(f"Test F1-score: {f1_3}")

In [ ]:
#@title Save model
model_name = "tl3_model" #@param {"type":"string"}
save_tf_model(tl3_model, models_dir, f'{model_name}.keras')

## 2.5 Compare Experiments

In [ ]:
#@title Results
# Group history objects
histories = [tl1_history, tl2_history, tl3_history]

# Compare training history
plot_comparison(histories, ['accuracy', 'val_accuracy', 'loss', 'val_loss'])  # Plot Accuracy and Validation Accuracy

# Group metrics
results = [
    [accuracy_1, precision_1, recall_1, f1_1],
    [accuracy_2, precision_2, recall_2, f1_2],
    [accuracy_3, precision_3, recall_3, f1_3]
]

# Compare test metrics
df_results = compare_test_results(results)
df_results


## 2.6 Save Models

In [ ]:
#@title Save

save_models = False #@param {type:"boolean"}
model_type_name = 'tl' #@param {"type":"string"}


if save_models:
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    tl1_model.save(os.path.join(models_dir, f'{model_type_name}_model_1_{timestamp}.keras'))
    tl2_model.save(os.path.join(models_dir, f'{model_type_name}_model_2_{timestamp}.keras'))
    tl3_model.save(os.path.join(models_dir, f'{model_type_name}_model_3_{timestamp}.keras'))


---
# Part 3 - Image Retrieval Experiments

## Configuration Analysis

The KNN experiments explore different neighborhood sizes for classification using embeddings from EfficientNetV2L:

1. **Small Neighborhood (k=3)**:
   - Tighter, more specific neighborhood
   - Higher sensitivity to local patterns
   - May be more precise but susceptible to noise
   - Best for distinct, well-separated classes

2. **Medium Neighborhood (k=5)**:
   - Balanced neighborhood size
   - Moderate smoothing of decision boundaries
   - Compromise between specificity and robustness
   - Good for general-purpose classification

3. **Large Neighborhood (k=10)**:
   - Broader neighborhood consideration
   - More robust to outliers
   - Smoother decision boundaries
   - Better for noisy or overlapping classes

Key differences:
- k=3 vs k=5: Tests impact of slightly larger neighborhood
- k=5 vs k=10: Evaluates benefit of much broader context
- k=3 vs k=10: Contrasts tight vs broad neighborhood effects

## 3.1 Set-up

In [ ]:
#@title Imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## 3.2 Embedding

KNN doesn't have a separate training phase where it learns parameters.

It simply memorizes the training data.

So we'll combine the training and validation data so we can learn from more data and hopefully get better results.

In [ ]:
#@title Embedding the data using EfficientNetV2 with Imagenet weights - same base model used for TL
def embed_data(embedding_model, dataset):
    """
        This function transforms the data to a format that is suited for the KNN prediction head
    """
    X, y = [], []
    for images, labels in dataset:
        image_embeddings = embedding_model.predict(images)
        # Flatten the embeddings if needed
        image_embeddings = image_embeddings.reshape(image_embeddings.shape[0], -1)
        X.extend(image_embeddings)
        y.extend(np.argmax(labels.numpy(), axis=1)) # Extract true class labels

    return np.array(X),  np.array(y)

# Embeddings
X_train_embed, y_train_embed = embed_data(base_model, train_ds)
X_val_embed, y_val_embed = embed_data(base_model, val_ds)
X_test_embed, y_test_embed = embed_data(base_model, test_ds)

# Combine X train and val since there is no weight learning in KNN we just memmorize
X_train_embed = np.concatenate([X_train_embed, X_val_embed])
y_train_embed = np.concatenate([y_train_embed, y_val_embed])

print(f"{X_train_embed.shape=} | {y_train_embed.shape=}")
print(f"{X_test_embed.shape=} | {y_test_embed.shape=}")

## 3.3 Experiments

In [ ]:
#@title Experiment 1 - k = 3
knn_1 = KNeighborsClassifier(n_neighbors=3)
knn_1.fit(X_train_embed, y_train_embed)
y_pred_test_1 = knn_1.predict(X_test_embed)

accuracy_1, precision_1, recall_1, f1_1 = test_model(y_pred_test_1, y_test_embed)

print("Experiment 1 (k=3):")
print(f"  Accuracy: {accuracy_1}")
print(f"  Precision: {precision_1}")
print(f"  Recall: {recall_1}")
print(f"  F1-score: {f1_1}")


In [ ]:
#@title Experiment 2 - k = 5
knn_2 = KNeighborsClassifier(n_neighbors=5)
knn_2.fit(X_train_embed, y_train_embed)
y_pred_test_2 = knn_2.predict(X_test_embed)

accuracy_2, precision_2, recall_2, f1_2 = test_model(y_pred_test_2, y_test_embed)

print("Experiment 2 (k=5):")
print(f"  Accuracy: {accuracy_2}")
print(f"  Precision: {precision_2}")
print(f"  Recall: {recall_2}")
print(f"  F1-score: {f1_2}")



In [ ]:
#@title Experiment 3 - k = 10
knn_3 = KNeighborsClassifier(n_neighbors=10)
knn_3.fit(X_train_embed, y_train_embed)
y_pred_test_3 = knn_3.predict(X_test_embed)

accuracy_3 = accuracy_score(y_test_embed, y_pred_test_3)
precision_3 = precision_score(y_test_embed, y_pred_test_3, average='macro')
recall_3 = recall_score(y_test_embed, y_pred_test_3, average='macro')
f1_3 = f1_score(y_test_embed, y_pred_test_3, average='macro')

print("Experiment 3 (k=10):")
print(f"  Accuracy: {accuracy_3}")
print(f"  Precision: {precision_3}")
print(f"  Recall: {recall_3}")
print(f"  F1-score: {f1_3}")


## 3.4 Compare Experiments

In [ ]:
# Group metrics
results = [
    [accuracy_1, precision_1, recall_1, f1_1],
    [accuracy_2, precision_2, recall_2, f1_2],
    [accuracy_3, precision_3, recall_3, f1_3]
]

# Compare test metrics
df_results = compare_test_results(results)
df_results

---
# Part 4 - E2E CNN Experiments

### 4.1 We chose ResNet type architecture for this experiment

Open for more info


**ResNets are known for their ability to train very deep networks effectively by using skip connections (residual blocks) to address the vanishing gradient problem.**

Here We define a simple ResNet (Residual Network) model for image classification.

**`residual_block` function:**

1. **`shortcut = x`:** Creates a copy of the input tensor `x` for the skip connection.

2. **Convolutional Layers:** Applies two convolutional layers with batch normalization and ReLU activation to the input.

3. **Shortcut Adjustment:** If necessary, adjusts the shortcut's dimensions using a 1x1 convolution and batch normalization to match the main processing path's output.

4. **`x = Add()([x, shortcut])`:** Performs element-wise addition of the main path output and the shortcut, implementing the residual connection.

5. **`x = Activation('relu')(x)`:** Applies ReLU activation to the sum.


**`create_resnet_model` function:**

1. **Input Layer:** Defines the input layer with the specified shape.

2. **Initial Layers:** Applies a convolutional layer, batch normalization, ReLU activation, and max pooling for initial feature extraction.

3. **Residual Blocks:** Adds a series of `residual_block`s to refine features.

4. **Global Average Pooling:** Reduces spatial dimensions using global average pooling.

5. **Output Layer:** Uses a dense layer with softmax activation for classification.

6. **Model Creation:** Creates a Keras model with the defined input and output layers.



### Configuration Analysis


The Custom ResNet experiments explore the impact of network depth through different numbers of residual blocks:

1. **Shallow Network (3 Blocks)**:
   - Minimal residual architecture
   - Faster training and inference
   - Lower model capacity
   - Tests if simple features are sufficient

2. **Medium Network (5 Blocks)**:
   - Balanced depth
   - Moderate feature hierarchy
   - Good capacity-complexity trade-off
   - Tests optimal depth for the task

3. **Deep Network (7 Blocks)**:
   - Maximum depth tested
   - Complex feature hierarchy
   - Highest model capacity
   - Tests benefits of deeper architecture

Key differences:
- 3 vs 5 blocks: Evaluates benefit of moderate depth increase
- 5 vs 7 blocks: Tests impact of further depth
- 3 vs 7 blocks: Contrasts shallow vs deep architectures

## 4.2 Set-up

In [ ]:
#@title Imports
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense


In [ ]:
#@title Model definition functions

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x

    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model(input_shape, num_classes, num_res_blocks):
    inputs = tf.keras.Input(shape=input_shape)

    # First conv-batch-relu-pool block as base layer feature extractor
    x = Conv2D(64, (7, 7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

    # Add res blocks for further refinement of features
    for _ in range(num_res_blocks):
        x = residual_block(x, 64)

    # AVG pool to further compress the results
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model



## 4.3 Expirement 1

In [ ]:
#@title Choose how many residual blocks to use
num_res_blocks = 3 #@param {type:"integer"}


In [ ]:
#@title Define model
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
cnn_model_1 = create_resnet_model(INPUT_SHAPE, NUM_CLASSES, num_res_blocks)

# Compile the model
cnn_model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
cnn_model_1.summary()


In [ ]:
#@title Load a pretrained model
#@markdown Loading from the set `models_dir`
# Load the pre-trained model
model_name = "cnn_model_1" # @param {"type":"string"}
model_path = models_dir + f'/{model_name}.keras' # Replace with the actual path to your saved model
tl1_model = tf.keras.models.load_model(model_path)


In [ ]:
#@title Train the model
cnn1_history = cnn_model_1.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)

In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(cnn1_history)

In [ ]:
#@title Test Model
accuracy_1, precision_1, recall_1, f1_1 = test_DL_model(cnn_model_1, test_ds)

print(f"Test Accuracy: {accuracy_1}")
print(f"Test Precision: {precision_1}")
print(f"Test Recall: {recall_1}")
print(f"Test F1-score: {f1_1}")

In [ ]:
#@title Save model
model_name = "cnn_model_1" #@param {"type":"string"}
save_tf_model(cnn_model_1, models_dir, f'{model_name}.keras')

## 4.4 Expirement 2

In [ ]:
#@title Choose how many residual blocks to use
num_res_blocks = 5 #@param {type:"integer"}


In [ ]:
#@title Define model
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
cnn_model_2 = create_resnet_model(INPUT_SHAPE, NUM_CLASSES, num_res_blocks)

# Compile the model
cnn_model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
cnn_model_2.summary()


In [ ]:
#@title Train the model
cnn2_history = cnn_model_2.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)


In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(cnn2_history)

In [ ]:
#@title Test Model
accuracy_2, precision_2, recall_2, f1_2 = test_DL_model(cnn_model_2, test_ds)

print(f"Test Accuracy: {accuracy_2}")
print(f"Test Precision: {precision_2}")
print(f"Test Recall: {recall_2}")
print(f"Test F1-score: {f1_2}")

In [ ]:
#@title Save model
model_name = "cnn_model_2" #@param {"type":"string"}
save_tf_model(cnn_model_2, models_dir, f'{model_name}.keras')

## 4.5 Expirement 3

In [ ]:
#@title Choose how many residual blocks to use
num_res_blocks = 7 #@param {type:"integer"}


In [ ]:
#@title Define model
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
cnn_model_3 = create_resnet_model(INPUT_SHAPE, NUM_CLASSES, num_res_blocks)

# Compile the model
cnn_model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
# cnn_model_3.summary()


In [ ]:
#@title Train the model
cnn3_history = cnn_model_3.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=1
)


In [ ]:
#@title Show training loss and accuracy
plot_accuracy_along_training(cnn3_history)

In [ ]:
#@title Test Model
accuracy_3, precision_3, recall_3, f1_3 = test_DL_model(cnn_model_3, test_ds)

print(f"Test Accuracy: {accuracy_3}")
print(f"Test Precision: {precision_3}")
print(f"Test Recall: {recall_3}")
print(f"Test F1-score: {f1_3}")

In [ ]:
#@title Save model
model_name = "cnn_model_3" #@param {"type":"string"}
save_tf_model(cnn_model_3, models_dir, f'{model_name}.keras')

## 4.6 Compare Experiments

In [ ]:
#@title Results
# Group history objects
histories = [cnn1_history, cnn2_history, cnn3_history]


# Compare training history
plot_comparison(histories, ['accuracy', 'val_accuracy', 'loss', 'val_loss'])  # Plot Accuracy and Validation Accuracy

# Group metrics
results = [
    [accuracy_1, precision_1, recall_1, f1_1],
    [accuracy_2, precision_2, recall_2, f1_2],
    [accuracy_3, precision_3, recall_3, f1_3]
]

# Compare test metrics
df_results = compare_test_results(results)
df_results


## 4.7 Save Models

In [ ]:
#@title Save

save_models = False #@param {type:"boolean"}
model_type_name = 'cnn' #@param {"type":"string"}


if save_models:
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    cnn_model_1.save(os.path.join(models_dir, f'{model_type_name}_model_1_{timestamp}.keras'))
    cnn_model_2.save(os.path.join(models_dir, f'{model_type_name}_model_2_{timestamp}.keras'))
    cnn_model_3.save(os.path.join(models_dir, f'{model_type_name}_model_3_{timestamp}.keras'))


# Part 5 - Test Environment

In [ ]:
#@title Imports
from google.colab import files
from PIL import Image

In [ ]:
#@title Load Best Model <font color="red"> - You need to set this upon your environment
best_model_path = '/path/to/best/model.keras'
best_model = tf.load_model(best_model_path)

In [ ]:
#@title Upload Files for prediction
uploaded = files.upload()

for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

    # Load the image
    img = Image.open(fn)

    # Preprocess the image (resize, normalize, etc.)
    # Adjust these parameters to match the preprocessing steps used during training
    img = img.resize((IMG_SIZE, IMG_SIZE)) # assuming IMG_SIZE is defined elsewhere in your code
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make the prediction
    prediction = best_model.predict(img_array)
    predicted_class = np.argmax(prediction)

    # Plot image
    plt.imshow(img)
    plt.title(f"Predicted Class: {predicted_class}")
    plt.axis('off')
    plt.show()

